In [ ]:
import functools
import itertools
import pathlib
from typing import *

import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import cv2 as cv
import pandas as pd
import matplotlib.pyplot as plt
from torchsummary import summary
from matplotlib import rc

%matplotlib inline

%load_ext autoreload
%autoreload 2

font = {'family': 'Times New Roman', 'weight': 'bold', 'size': 12}
rc('font', **font)

In [ ]:
import json

def make_crowdhuman_iter(box_type='fbox'):
    def _iter_crowdhuman_annos(anno_file_path):
        with open(anno_file_path) as anno_file:
            for anno_entry in map(json.loads, anno_file.readlines()):
                image_id = anno_entry['ID']
                gt_boxes = anno_entry['gtboxes']

                for box_entry in gt_boxes[box_type]:
                    yield image_id, box_entry
    
    return _iter_crowdhuman_annos



In [ ]:
val = 'ff'
int(val, base=16)
                # format: "non-unique int val,unique hex val"
                image_id_tokens = anno_entry['ID'].split(',')
                image_id = int(image_id_tokens[1], base=16)